In [1]:
%matplotlib ipympl
from QDP import qdp, atom
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize

In [2]:
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exclude = [2]
exp_name = '2018_07_26_15_08_02_temperature-scan-8.5-20x'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)

cs_cuts = [[33],[3]]
rb_cuts = [[32],[3]]
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
# q.generate_thresholds(loss=False)
retention = q.apply_thresholds(exclude_rois=exclude, ncondition=1)

xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
print(xlab)

data at: 2018_07_26/2018_07_26_15_08_02_temperature-scan-8.5-20x/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
fort off time (us)


/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:131: RuntimeWarning: overflow encountered in double_scalars
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:131: RuntimeWarning: invalid value encountered in sqrt
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))


In [28]:
fig, ax = plt.subplots()

ax.set_ylim(0,1)
for i in range(len(retention['ivar'][0,0])/11):
    ax.plot(retention['ivar'][0,0,:11], retention['conditional_retention'][0,:,0].reshape(-1,11)[i], linestyle='None', marker='o')

FigureCanvasNbAgg()

In [30]:
fig, ax = plt.subplots()
ax.set_ylim(0,1)
for i in range(len(retention['ivar'][0,0])/11):
    ax.plot(retention['ivar'][0,0,:11], retention['conditional_retention'][0,:,1].reshape(-1,11)[i], linestyle='None', marker='d')

FigureCanvasNbAgg()

In [49]:
q.save_experiment_data(filename_prefix='cooling_attempt_data')

In [75]:
def fnode_release_recapture(t, T_uk, a='cs'):
    wr_um = 2.4
    zr_um = 17  # calculated
    pfort_nom = 90.  # 80 mV on detector
    pfort_act = 84  # mV on detector
    fort_depth_fac = pfort_act/pfort_nom
    if a == 'cs':
        U_mk = 1.27*fort_depth_fac
        fr_khz = 37.4*np.sqrt(fort_depth_fac)
        fa_khz = 3.730*np.sqrt(fort_depth_fac)
        m_au = 132.905
    else:  # rb
        U_mk = 0.744*fort_depth_fac
        fr_khz = 35.4*np.sqrt(fort_depth_fac)
        fa_khz = 3.57*np.sqrt(fort_depth_fac)
        m_au = 86.909
    n=50000
    try:
        res = np.zeros(len(t))
        for i in xrange(len(t)):
            res[i] = atom.release_recapture(t[i], T_uk, U_mk, wr_um, zr_um, fr_khz, fa_khz, n=n, m_au=m_au)
        return res
    except Exception as e:
        print(e)
        return atom.release_recapture(t, T_uk, U_mk, wr_um, zr_um, fr_khz, fa_khz, n=n, m_au=m_au)

def residuals(x, y, t, e, a='cs'):
    T_uk = x[0]
    f0 = x[1]
    res = (y - f0*fnode_release_recapture(t, T_uk, a=a))*np.power(e,-2)
    #print "T_uk = {}, -> {}".format(T_uk, np.sum(np.power(res, 2)))
    return res

def ls_err(x, y, t, f0, w, a='cs'):
    res = residuals(T_uk, y, t, f0, a=a)
    return np.dot(np.power(res, 2), w)

In [77]:
fig, ax = plt.subplots()
T_opts = []
T_uncerts = []
T_opt = 90.  # initial temperature guess

for e in range(len(retention['ivar'][0,0])/11):
    ax.errorbar(retention['ivar'][0,0,:11], retention['conditional_retention'][0,:,0].reshape(-1,11)[e],
                yerr=retention['conditional_retention_err'][0,:,0].reshape(-1,11)[e],
                fmt = 'o'
    )
    p0 = np.mean(retention['conditional_retention'][0,:,0].reshape(-1,11)[e,:2])
    guess = (T_opt, p0)
    x = optimize.least_squares(
        residuals,
        guess,
        args=(retention['conditional_retention'][0,:,0].reshape(-1,11)[e], retention['ivar'][0,0,:11], retention['conditional_retention_err'][0,:,0].reshape(-1,11)[e]),
        diff_step=0.05,
        xtol=0.01
    )
    if x['success']:
        T_opt = x['x'][0]
    T_opts.append(T_opt)
    
    J = x.jac
    res = residuals(x['x'], retention['conditional_retention'][0,:,0].reshape(-1,11)[e], retention['ivar'][0,0,:11], retention['conditional_retention_err'][0,:,0].reshape(-1,11)[e])
    ucert = np.sqrt(np.abs(np.linalg.inv(J.T.dot(J)) * (res**2).mean())[0,0])
    T_uncerts.append(ucert)
    print ucert

    xs = np.linspace(0, 26, 30)
    ax.plot(xs, fnode_release_recapture(xs, T_opt)*x['x'][1], 'k-')
    ax.text(1, 0.4-e*0.07, "T = {:.1f} uK".format(T_opt))
    
ax.set_ylim(0, 1.05)
ax.set_xlabel(xlab)
ax.set_ylabel('retention')

FigureCanvasNbAgg()

7.22127708342
7.44868606849
4.73896511895
6.35419630979
5.18872311982
13.1179225094
6.3662879227
5.20776142827
7.27973207396
5.26702370539
5.35874085664
7.22830076938
8.16823704547
5.51693193001
8.03535372299
9.37492624512
7.89216626505
8.56444619065
8.91497936479
6.15758222582


Text(0,0.5,u'retention')

In [78]:
fig, ax = plt.subplots()

ax.errorbar(range(20), T_opts, yerr=T_uncerts, fmt='bo')
ax.set_ylim(0,150)

FigureCanvasNbAgg()

(0, 150)

In [79]:
fig, ax = plt.subplots()
guess = 90
Tr_opts = []
Tr_uncerts = []
T_opt = 90.

for e in range(len(retention['ivar'][0,0])/11):
    ax.errorbar(retention['ivar'][0,0,:11], retention['conditional_retention'][0,:,1].reshape(-1,11)[e],
                yerr=retention['conditional_retention_err'][0,:,1].reshape(-1,11)[e],
                fmt = 'o'
    )
    p0 = np.mean(retention['conditional_retention'][0,:,1].reshape(-1,11)[e,:2])
    guess = (T_opt, p0)
    x = optimize.least_squares(
        residuals,
        guess,
        args=(retention['conditional_retention'][0,:,1].reshape(-1,11)[e], retention['ivar'][0,0,:11], retention['conditional_retention_err'][0,:,1].reshape(-1,11)[e], 'rb'),
        diff_step=0.05,
        xtol=0.01
    )
    if x['success']:
        T_opt = x['x'][0]
    Tr_opts.append(T_opt)
    
    J = x.jac
    res = residuals(x['x'], retention['conditional_retention'][0,:,0].reshape(-1,11)[e], retention['ivar'][0,0,:11], retention['conditional_retention_err'][0,:,0].reshape(-1,11)[e])
    ucert = np.sqrt(np.abs(np.linalg.inv(J.T.dot(J)) * (res**2).mean())[0,0])
    Tr_uncerts.append(ucert)
    print ucert
    
    xs = np.linspace(0, 26, 30)
    ax.plot(xs, fnode_release_recapture(xs, T_opt, a='rb')*x['x'][1], 'k-')
    ax.text(1, 0.4-e*0.07, "T = {:.1f} uK".format(T_opt))
    
ax.set_ylim(0, 1.05)
ax.set_xlabel(xlab)
ax.set_ylabel('retention')

FigureCanvasNbAgg()

12.4155929049
6.36641064529
8.05396155245
6.27896239624
5.27795224061
9.58433408728
4.02493189406
5.2560328734
5.10343414402
8.03893030686
6.92242151574
6.69134079001
8.33597797613
6.42579992651
8.0727435663
9.53921687569
6.67998143419
14.2994419248
10.4999752429
6.04971044094


Text(0,0.5,u'retention')

In [80]:
fig, ax = plt.subplots(figsize=(2*3.375,3.5))

marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='--'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='--')
]

ax.errorbar(range(20), T_opts, yerr=T_uncerts, **marker_styles[0])
ax.errorbar(range(20), Tr_opts, yerr=Tr_uncerts, **marker_styles[1])
ax.set_ylim(0,120)
ax.grid()
ax.set_ylabel(r'Temperature ($\mu$K)', fontsize=14)
ax.set_xlabel('Trial Number', fontsize=14)
plt.tight_layout()

fmts = ['pdf', 'png', 'eps']
out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/temp-stability/'
fn = os.path.join(out, 'temp-stability.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

In [83]:
print(np.max(T_opts))
print(np.min(T_opts))

98.4551812726
64.8280347474


In [84]:
print(np.max(Tr_opts))
print(np.min(Tr_opts))

71.7688446621
46.7819155895
